# USA accidents EDA


Data Source : Kaggle, Information about accidents

Use : can be useful to prevent accidents


## Initialization & Download data

1.Load the file using Pandas

2.look at some information about the data & the data the column

In [ ]:
import numpy as np 
import pandas as pd
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
        

In [ ]:
data = pd.read_csv('/kaggle/input/us-accidents/US_Accidents_March23.csv')

In [ ]:
import seaborn as sns
sns.set_style('darkgrid')
import matplotlib.pyplot as plt
%matplotlib inline
import plotly.graph_objects as go
import plotly
import plotly.offline as iplot
import plotly.express as px
import plotly.graph_objects as go


In [ ]:
data.head()

In [ ]:
data.columns

In [ ]:
len(data.columns)

In [ ]:
data.info()

In [ ]:
data.describe()

Most of the accidents seems a point accidents, due to lower number of end latitude and end time values

In [ ]:
numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
numdata = data.select_dtypes(include=numerics)
len(numdata.columns)

# Data Preparation and cleaning

- Check missing or incorrect values

Missing values and thier percentage

In [ ]:
Miss_data_perc = (100* data.isna().sum().sort_values(ascending = False)/len(data))
Miss_data_perc[Miss_data_perc != 0]

In [ ]:
Miss_data_perc[Miss_data_perc > 25]

removing data where there are more thn 25% of missing data, and country column, for further analysis

In [ ]:
#cleaning data
dataC = data[[col for col in data.columns if col not in ['End_Lat','End_Lng', 'Precipitation(in)', 'Wind_Chill(F)', 'Country']]]

Check if all the states are mentioned

In [ ]:
info_states = dataC.State.unique()
len(dataC.State.unique())

In [ ]:
#list of all states in (source:https://snipplr.com/view/50728/list-of-us-state-abbreviations)
states = ["AL", "AK", "AZ", "AR", "CA", "CO", "CT", "DC", "DE", "FL", "GA", 
              "HI", "ID", "IL", "IN", "IA", "KS", "KY", "LA", "ME", "MD", 
              "MA", "MI", "MN", "MS", "MO", "MT", "NE", "NV", "NH", "NJ", 
              "NM", "NY", "NC", "ND", "OH", "OK", "OR", "PA", "RI", "SC", 
              "SD", "TN", "TX", "UT", "VT", "VA", "WA", "WV", "WI", "WY"]
len(states)

In [ ]:
no_info_states =[]
for i in states:
    if i not in info_states:
        no_info_states.append(i)
print(no_info_states)   

The used data do not have an information from Alaska and Hawaii

#### Converting bool columns

In [ ]:
bool_columns = ['Amenity', 'Bump', 'Crossing', 'Give_Way', 'Junction', 'No_Exit', 'Railway', 'Roundabout', 'Station', 'Stop', 'Traffic_Calming', 'Traffic_Signal', 'Turning_Loop']

for col in bool_columns:
    dataC[col] = dataC[col].astype(float)
dataC.info()

# Exploratory analysis and visulazation

cloumns to analyse:

1.City

2.Weekly, yearly, daily

3.Location, aka ; Start lat, start lng

4.Temprature

5.Weather Condition

### City wise EDA


In [ ]:
cites = dataC.City.unique()
len(cites)

In [ ]:
CITYwrtACi = dataC["City"].value_counts()
CITYwrtACi[:40].plot(kind='bar')

In [ ]:
sns.displot(CITYwrtACi, kde = True, bins = 200)

observation: large gap in data: dividing the data in to sections

In [ ]:
sns.histplot(CITYwrtACi,log_scale = True, bins = 80)

Most of the cityes have accidents ranging from around 1 to 100

In [ ]:
CITYwrtACi[CITYwrtACi == 1]

Aleast 1000 cities has reported 1 accident

In [ ]:
CITYwrtACi[CITYwrtACi< 10]

we have almost 4250 cities with less than 10 accidents  

In [ ]:
high_acci_city = CITYwrtACi[CITYwrtACi>1000]
low_acci_city = CITYwrtACi[CITYwrtACi<=1000]
print("low accident city % :",(len(low_acci_city)/len(CITYwrtACi))*100)
print("High accident city % :",(len(high_acci_city)/len(CITYwrtACi))*100)

In [ ]:
CITYwrtACi.describe()

## Weekly, yearly, daily (Start time)

In [ ]:
type(dataC.Start_Time[0])

In [ ]:
dataC.Start_Time.head()

In [ ]:
dataC.Start_Time = pd.to_datetime(dataC.Start_Time)

In [ ]:
type(dataC.Start_Time[0])

In [ ]:
sns.displot(dataC.Start_Time.dt.hour, bins = 24, stat='probability')

there is a peak in values in betwee, 7:00 hours to 9:00 hours, and another spike between 15:00 hours to 18:00 hours

In [ ]:
sns.displot(dataC.Start_Time.dt.dayofweek,bins=7, stat='probability')

On weekeds number of accidents is lower
id the disribution of accidents by hour the same on the weekend as on weekdays

In [ ]:
Saturday_ST = dataC.Start_Time[dataC.Start_Time.dt.dayofweek == 6]
sns.displot(Saturday_ST.dt.hour, bins = 24, stat='probability')

In [ ]:
Sunday_ST = dataC.Start_Time[dataC.Start_Time.dt.dayofweek == 6]
sns.displot(Sunday_ST.dt.hour, bins = 24, stat='probability')

Sunday and saturday shows almost same type of disribution, peak number of accidents in the afternoon hours and more later i n the evening compared to morning, maybe because people generally wake up late during moring and return late at night

In [ ]:
Monday_ST = dataC.Start_Time[dataC.Start_Time.dt.dayofweek == 0]
sns.displot(Monday_ST.dt.hour, bins = 24, stat='probability')

Moday Has almost the same graph of the average daily distribution

In [ ]:
sns.histplot(dataC.Start_Time.dt.month, bins = 12)

we can see some peak during winter months, might be due to lower visibility and slippery roads compared to summer time

In [ ]:
sns.displot(dataC.Start_Time.dt.year,)

we cannot relay on data of 2023, because currenty it is 2023. There is steady increse in accidents thoughout the years, even during the 2020, 2021 pandamic period, maybe due to lower relability on public transports and incresed car or individula vehicle useage. Might also be due to lower collection of data in previours years

## Source

In [ ]:
dataC['Source']

In [ ]:
dataC.Source.value_counts().plot(kind='pie')

The sources are not named here, from the discussion section of the dataset it seems like the sources are mainly from Bing and MapQuest

## States in States

In [ ]:
dataC.columns

In [ ]:
state_accidents = dataC['State'].value_counts()
state_accidents = pd.DataFrame({'State': state_accidents.index, 'Accidents': state_accidents.values})

In [ ]:
Count_Accl_Sta = pd.DataFrame(dataC['State'].value_counts())
z = Count_Accl_Sta.values.flatten()
x = Count_Accl_Sta.index.to_list()
fig = go.Figure(data = go.Choropleth(locations = x, z = z, locationmode = "USA-states", colorscale = 'plasma'))

fig.update_layout(title_text = "Number of Accidents for each State in US", geo_scope = "usa")
fig.show()

In [ ]:
fig = px.bar(Count_Accl_Sta[0:15], x=Count_Accl_Sta[0:15].index, y=Count_Accl_Sta[0:15].values.flatten(), 
             color=Count_Accl_Sta[0:15].index, labels={'x': 'States', 'y': 'Number of Accidents'})
fig.update_layout(
    title="Top 15 States with Highest Number of Accidents",
    xaxis_tickangle=-90,
    yaxis_title="Number of Accidents",
    xaxis_title="States",
    plot_bgcolor='rgba(0,0,0,0)'
)
fig.show()

In [ ]:
city_acc_counts = pd.DataFrame(dataC['City'].value_counts()).reset_index()
city_acc_counts.columns = ['City',"Number of Accidents"]
city_acc_counts.sort_values(by = 'Number of Accidents', ascending = False,inplace = True)
x = city_acc_counts['City'][:15].to_list()
y = city_acc_counts["Number of Accidents"][:15]

In [ ]:
fig = go.Figure(data=go.Bar(x=x, y=y, marker_color='mediumaquamarine'))
fig.update_layout(
    title="Top 15 Cities with Highest Number of Accidents",
    xaxis_tickangle=-90,
    yaxis_type="log",
    yaxis_title="Number of Accidents",
    xaxis_title="Cities",
    plot_bgcolor='rgba(0,0,0,0)'
)
fig.show()

 ## Temprature

In [ ]:
sns.displot(dataC['Temperature(F)'],bins = 50,kde= True)

In [ ]:
dataC['Temperature(F)'].mean()

In [ ]:
dataC['Temperature(F)'].value_counts().head(1) # or mode

Most of the accidents happen when the temprature was round 77°F that is 25°C, which does not tally with the information most of the accident data is belong winter months, and might be due to the fact 25°C is usuallay taken as environment temprature or room temprature by defalut

## Weather

In [ ]:
Top_accI_wea = dataC["Weather_Condition"].value_counts()[:25]
Top_accI_wea

In [ ]:
plt.figure(figsize=(20, 8))
sns.barplot(x = Top_accI_wea.index, y=Top_accI_wea.values )

In [ ]:
dataC["Weather_Condition"].unique()

even though most of the accidents are shown for a "Fine day", many rows assosiated with rain, windy,snow

In [ ]:
rain_related = ['Light Rain', 'Rain', 'Heavy Rain', 'Light Rain Showers', 'Rain Showers', 'Heavy Rain Showers', 'Heavy Rain / Windy', 'Thunderstorms and Rain', 'Thunderstorm', 'T-Storm', 'Heavy Thunderstorms and Rain', 'Thunder', 'T-Storm / Windy', 'Thunder / Windy', 'Thunder and Hail', 'Thunder and Hail / Windy']

cloud_related = ['Overcast', 'Mostly Cloudy', 'Cloudy', 'Scattered Clouds', 'Partly Cloudy', 'Mostly Cloudy / Windy', 'Partly Cloudy / Windy', 'Cloudy / Windy','Wintry Mix / Windy', 'Sleet', 'Light Sleet', 'Sleet / Windy', 'Heavy Sleet', 'Heavy Sleet / Windy']

snow_related = ['Light Snow', 'Snow', 'Light Snow Shower', 'Snow Showers', 'Heavy Snow', 'Heavy Snow / Windy', 'Light Snow / Windy', 'Snow / Windy', 'Snow and Sleet', 'Snow and Sleet / Windy', 'Heavy Thunderstorms and Snow']

clear = ['Clear', 'Fair', 'Fair / Windy']

drizzle_fog_related = ['Light Drizzle', 'Drizzle', 'Light Freezing Drizzle', 'Heavy Drizzle', 'Drizzle and Fog', 'Mist', 'Fog', 'Shallow Fog', 'Fog / Windy', 'Shallow Fog / Windy', 'Drizzle / Windy', 'Light Drizzle / Windy', 'Light Freezing Rain', 'Freezing Drizzle', 'Light Freezing Rain / Windy', 'Freezing Rain', 'Heavy Freezing Drizzle', 'Heavy Freezing Rain', 'Freezing Rain / Windy', 'Heavy Freezing Rain / Windy']

haze_smoke_related = ['Haze', 'Smoke', 'Light Haze', 'Smoke / Windy', 'Haze / Windy','Sand', 'Blowing Sand', 'Sand / Dust Whirlwinds', 'Sand / Dust Whirlwinds / Windy', 'Duststorm', 'Dust Whirls', 'Blowing Dust', 'Blowing Dust / Windy', 'Widespread Dust', 'Widespread Dust / Windy', 'Squalls / Windy', 'Dust Whirls Nearby', 'Blowing Snow', 'Blowing Snow / Windy', 'Blowing Snow Nearby']

other = ['Light Freezing Fog', 'Light Ice Pellets', 'Ice Pellets', 'Small Hail', 'Wintry Mix', 'Squalls', 'Drifting Snow', 'Low Drifting Snow', 'Drifting Snow']

In [ ]:
def map_weather_condition(condition):
    if condition in rain_related:
        return 'Rain_related'
    elif condition in cloud_related:
        return 'Cloudy'
    elif condition in snow_related:
        return 'Snow_related'
    elif condition in clear:
        return 'Clear'
    elif condition in drizzle_fog_related:
        return 'Drizzle_Fog_related'
    elif condition in haze_smoke_related:
        return 'Haze_Smoke_related'
    else:
        return 'Other'

# Apply the mapping function to create the new column
dataC['Weather_Category'] = dataC['Weather_Condition'].map(map_weather_condition)
dataC.head(1)

In [ ]:
category_counts = dataC['Weather_Category'].value_counts()
category_percents = category_counts/len(dataC)*100
fig = px.pie(category_percents, names = category_percents.index, values = category_percents.values, title = "Weather Category Percentage") 
fig.show()

Most of the accidents happen during a relativly Clear day and second most on cloudy days.

## Severity

In [ ]:
dataC["Severity"].unique()

There are 4 level of Severity , with 1 for lowest impact and 4 for highest

In [ ]:
severity_df = pd.DataFrame(dataC['Severity'].value_counts()).rename(columns={'index':'Severity', 'Severity':'Cases'})


fig = go.Figure(go.Funnelarea(
    text = ["Severity - 2","Severity - 3", "Severity - 4", "Severity - 1"],
    values = severity_df.Cases,
    title = {"position": "top center", 
             "text": "<b>Impact on the Traffic due to the Accidents</b>", 
             'font':dict(size=18,color="#7f7f7f")},
    marker = {"colors": ['#14a3ee', '#b4e6ee', '#fdf4b8', '#ff4f4e'],
                "line": {"color": ["#e8e8e8", "wheat", "wheat", "wheat"], "width": [7, 0, 0, 2]}}
    ))

fig.show()



In 80% Cases of road accidents, the impact on the traffic was Moderate (Severity-2).
In 7.5% Cases of road accidents, the impact on the traffic was highly Severe (Severity-4).

In [ ]:
x_cols = [col for col in dataC.columns if col not in ['Severity'] if dataC[col].dtype == 'float64']

labels = []
values = []
for col in x_cols:
    labels.append(col)
    values.append(np.corrcoef(dataC[col].values, dataC.Severity.values)[0, 1])

corr_df = pd.DataFrame({'col_labels': labels, 'corr_values': values})
corr_df = corr_df.sort_values(by='corr_values')

fig = go.Figure(data=[go.Bar(
    x=corr_df.corr_values,
    y=corr_df.col_labels,
    orientation='h',
    marker=dict(color='yellow'),
)])

fig.update_layout(
    title="Correlation coefficient of the variables",
    xaxis=dict(title="Correlation coefficient"),
    yaxis=dict(automargin=True),
    height=800,
    width=600,
)

fig.show()


### Temprature Vs Severity

In [ ]:

fig = px.scatter(dataC, x='Temperature(F)', y='Severity', color='Severity', title='Temperature vs Severity',
                 labels={'Temperature(F)': 'Temperature (F)', 'Severity': 'Severity'})

fig.update_traces(marker=dict(size=8), selector=dict(mode='markers'))
fig.update_layout(coloraxis=dict(colorscale='Viridis'))

fig.show()


### Severity vs day of the week

In [ ]:
weekday_labels = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']
dataC['Weekday'] = dataC.Start_Time.dt.dayofweek.map(dict(zip(range(7), weekday_labels)))
weekDay_dict = {'Monday':{1:0,2:0,3:0,4:0}, 'Tuesday':{1:0,2:0,3:0,4:0}, 'Wednesday':{1:0,2:0,3:0,4:0}, 'Thursday':{1:0,2:0,3:0,4:0}, 'Friday':{1:0,2:0,3:0,4:0}, 'Saturday':{1:0,2:0,3:0,4:0}, 'Sunday':{1:0,2:0,3:0,4:0}}

In [ ]:
for index, row in dataC.iterrows():
    weekday = row['Weekday']
    severity = row['Severity']
    
    # Update the weekDay_dict count based on the weekday and severity
    weekDay_dict[weekday][severity] += 1

print(weekDay_dict)

In [ ]:
weekdays = list(weekDay_dict.keys())
severities = sorted(list(set(severity for values in weekDay_dict.values() for severity in values.keys())))

data = []
for severity in severities:
    counts = [weekDay_dict[weekday][severity] for weekday in weekdays]
    trace = go.Bar(
        x=weekdays,
        y=counts,
        name=f"Severity {severity}"
    )
    data.append(trace)

layout = go.Layout(
    barmode='group',
    title='Accident Counts by Weekday and Severity',
    xaxis=dict(title='Weekday'),
    yaxis=dict(title='Count')
)

fig = go.Figure(data=data, layout=layout)

fig.show()

* We can see that most of the sevier cases, except for Sevierty 1, increses from Monday to Friday, then sharp drop in numbers during the weekends, decline from Saturday to Sunday

## Distance

In [ ]:
fig = go.Figure(data=go.Scatter(x=dataC.index, y=dataC['Distance(mi)'], mode='markers'))

fig.update_layout(title='Distances of Accidents',
                  xaxis_title='Index',
                  yaxis_title='Distance (mi)')

fig.show()

## Road conditions and Warning

In [ ]:
columns = ['Amenity', 'Bump', 'Crossing', 'Give_Way', 'Junction', 'No_Exit', 'Railway',
           'Roundabout', 'Station', 'Stop', 'Traffic_Calming', 'Traffic_Signal', 'Turning_Loop']

counts = dataC[columns].sum()


In [ ]:
fig = go.Figure(data=[go.Bar(x=counts.index, y=counts.values)])

fig.update_layout(title='Count of True Values (1) in Each Column',
                  xaxis_title='Columns',
                  yaxis_title='Count')

fig.show()


## Night and Day

In [ ]:
dataC['Civil_Twilight'].unique()

In [ ]:
twilight_counts = dataC['Civil_Twilight'].value_counts()

fig = go.Figure(data=[go.Bar(x=twilight_counts.index, y=twilight_counts.values)])

fig.update_layout(title='Count of Day and Night',
                  xaxis_title=' Day or Night Accidents',
                  yaxis_title='Count')
fig.show()

* There are more accidents during Daytime, might be because of more footfall during those hours

# Questions and Answers

1. What is the severity of accidents in most cases in the given dataset?
* Answer: The average severity of accidents is 2.


2. What is the most common weather condition during accidents?
* Answer: The most common weather condition during accidents is "Clear Weather" with a frequency of 44.1% of cases, closly followed by "Cloud_related" weather Condition in 40.9% of cases.


3. Which state has the highest number of accidents?
* Answer: State California has the highest number of accidents with a count of 1.7 M followed by Florida with above 880k.


4. What is the average distance of accidents in miles?
* Answer: 


5. How many accidents occur during daytime versus nighttime?
* Answer: Most of the accidents happen during Day time, there is almost 3.5 Million cases more during day time when compared to Night

6. Which city has the highest number of accidents?
* Answer: City of Miami has the highest number of accidents with a count of 186.91k cases.

7. How many accidents involve traffic signals or stop signs?
* Answer:

8. Is there a significant difference in accident severity between weekdays and weekends?
* Answer: Yes, accidents on weekdays tend to have a higher severity compared to weekends, most Number of very Sevier cases happen mostly on Friday

# Summay and conclusion

Insights
- No data from Alaska and Hawai
- The number of accidents per city show exponental trend
- Less than 9% of cities have accidents more than 1000
- Almost 4250 cities with less than 10 accidents 
- High peercentage of accidents occure between 7am-9am and 3-5 pm ( work starting and ending time)